# INFO 212: Data Science Programming 1
___

### Week 6: Data Loading, Storage, and File Formats
___

### Mon., May 7, and Wed., May 9, 2018
---

**Question:**
- How to interact with external data sources using Python? 

**Objectives:**
- Read and write data from text formats
- Read and write JSON data
- Scrape data from the Web
- Merge multiple extracted data sets

Accessing data is a necessary first step for data analysis. We are going to be focused on data input and output using pandas, though there are numerous
tools in other libraries to help with reading and writing data in various formats.
Input and output typically falls into a few main categories: reading text files and other more efficient on-disk formats, loading data from databases, and interacting with network sources like web APIs.

In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)
%matplotlib inline

## Data Slicing
Slicing, selecting, and filtering pieces of data from a large structure is a fundamental skill for data analysis in Python. Here are some rules:
- forward slicing goes left to right and top to bottom;
- by default, it is forward slicing; backward slicing indicated by a negative third number;
- each position can be identify by a positive index (forward view) and a negative index (backward view); both indicies are equivalent;
- if the second index appears, the content at the second index is not included (right open);
- two ends (outside) are specified by nothing; so if an index is missing, the system automatically figure it out whether it is at the beginning or end based on its relation to another index;
- forward slicing: the index at the first position is before the index of the second position
- backward slicing: the index at the first postion is before (backwardly) the index of the second position.

```
s = [7, 2, 5, 6, 0, 1, 4]```

```
s[1:5]```

```
s[-6:-2]```

```
s[1:-2]```

```
s[-6:5]
```

## Reading and Writing Data in Text Format
pandas features a number of functions for reading tabular data as a DataFrame
object. The methods `read_csv` and `read_table` are likely the ones we’ll use the most.

```
!cat examples/ex1.csv
```

```
df = pd.read_csv('examples/ex1.csv')
df
```

How to use `read_table()` to read a csv file?

```
pd.read_table('examples/ex1.csv', sep=',')
```

How to read a csv file without a header row?

```
!cat examples/ex2.csv
```

Let the Pandas to assign default header:

```
pd.read_csv('examples/ex2.csv', header=None)
```

Or specify your own header:

```
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('examples/ex2.csv', names=names)```

The parser functions have many additional arguments to help you handle the wide
variety of exception file formats that occur. For example, you can skip the first, third, and fourth rows of a file with skiprows:

```
!cat examples/ex4.csv
```

```
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])```

How to handle missing data in the input file?

```
!cat examples/ex5.csv
```

```
result = pd.read_csv('examples/ex5.csv')
result
```

```
pd.isnull(result)
```

You can specify `na_values` for treating missing value:

```
result = pd.read_csv('examples/ex5.csv', na_values=['null'])
result
```

Different NA sentinels can be specified for each column in a dict:

```
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)
```

### Reading Text Files in Pieces
How to read a certain number of lines from a file?

```
!wc  -l examples/ex6.csv
```

```
pd.read_csv('examples/ex6.csv', nrows=5)
```

### Writing Data to Text Format
Data can also be exported to a delimited format.

```
data = pd.read_csv('examples/ex5.csv')
data
```

```
data.to_csv('examples/out.csv')
!cat examples/out.csv```

How to write csv file using different deliminter?

```
import sys
data.to_csv(sys.stdout, sep='|')
```

How to write null values explicitly?

```
data.to_csv(sys.stdout, na_rep='NULL')
```

How to ignore index and header labels in the written file?

```
data.to_csv(sys.stdout, index=False, header=False)
```

How to write out a subset of the columns?

```
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])
```

### JSON Data
JSON (short for JavaScript Object Notation) has become one of the standard formats
for sending data by HTTP request between web browsers and other applications. It is
a much more free-form data format than a tabular text form like CSV. Here is an
example.

In [38]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

How to parse a JSON structure into a Python object?

```
import json
result = json.loads(obj)
type(result)
```

```
result```

How to convert a Python object back to JSON?

```
asjson = json.dumps(result)
asjson```

```
type(asjson)```

How you convert a JSON object or list of objects to a DataFrame or some other data
structure for analysis will be up to you. Conveniently, you can pass a list of dicts
(which were previously JSON objects) to the DataFrame constructor and select a subset
of the data fields.

```
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings```

The pandas `read_json` can automatically convert JSON datasets in specific arrangements
into a Series or DataFrame.

```
!cat examples/example.json```

```
data = pd.read_json('examples/example.json')
data```

```
print(data.to_json())
print(data.to_json(orient='records'))```

### XML and HTML: Web Scraping
Python has many libraries for reading and writing data in the ubiquitous HTML and
XML formats. Examples include lxml, Beautiful Soup, and html5lib. While lxml is
comparatively much faster in general, the other libraries can better handle malformed
HTML or XML files.

Pandas has a built-in function, read_html, which uses libraries like lxml and Beautiful
Soup to automatically parse tables out of HTML files as DataFrame objects. To
show how this works, I downloaded an HTML file (used in the pandas documentation)
from the United States FDIC government agency showing bank failures.1 First,
you must install some additional libraries used by read_html:

conda install lxml
pip install beautifulsoup4 html5lib

```
tables = pd.read_html('examples/fdic_failed_bank_list.html')
len(tables)```

```
failures = tables[0]
failures.head()```

```
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()```

## Binary Data Formats

One of the easiest ways to store data (also known as serialization) efficiently in binary format is using Python’s built-in pickle serialization. pandas objects all have a
to_pickle method that writes the data to disk in pickle format:

```
frame = pd.read_csv('examples/ex1.csv')
frame```

```
frame.to_pickle('examples/frame_pickle')```

```
pd.read_pickle('examples/frame_pickle')```

```
!rm examples/frame_pickle```

### Reading Microsoft Excel Files

pandas also supports reading tabular data stored in Excel 2003 (and higher) files
using either the ExcelFile class or pandas.read_excel function. Internally these
tools use the add-on packages xlrd and openpyxl to read XLS and XLSX files, respectively.

To use ExcelFile, create an instance by passing a path to an xls or xlsx file:

```
xlsx = pd.ExcelFile('examples/ex1.xlsx')```

Data stored in a sheet can then be read into DataFrame with parse:

```
pd.read_excel(xlsx, 'Sheet1')```

If you are reading multiple sheets in a file, then it is faster to create the ExcelFile,
but you can also simply pass the filename to pandas.read_excel:

```
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame```

To write pandas data to Excel format, you must first create an ExcelWriter, then
write data to it using pandas objects’ to_excel method:

```
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()```

You can also pass a file path to to_excel and avoid the ExcelWriter:

```
frame.to_excel('examples/ex2.xlsx')```

```
!rm examples/ex2.xlsx```

## Interacting with Web APIs

Many websites have public APIs providing data feeds via JSON or some other format.
There are a number of ways to access these APIs from Python; one easy-to-use
method recommended is the requests package.

To find the last 30 GitHub issues for pandas on GitHub, we can make a GET HTTP
request using the add-on requests library:

```
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp```

The Response object’s json method will return a dictionary containing JSON parsed
into native Python objects:

```
data = resp.json()
data[0]['title']```

Each element in data is a dictionary containing all of the data found on a GitHub
issue page (except for the comments). We can pass data directly to DataFrame and
extract fields of interest:

```
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues
```

## Combine and Merge Data Sets

After we gathered data from external sources, it may be necessary to combine data in different sets. Data contained in pandas objects can be combined together in a number of ways:
- pandas.merge connects rows in DataFrames based on one or more keys. 
- pandas.concat concatenates or “stacks” together objects along an axis.
- The combine_first instance method enables splicing together overlapping data to fill in missing values in one object with values from another.

Here we will consider concatenating data frames along an axis for the purpose of assignment 3.

The pandas `pandas.concat(objs, axis=0, ...)` function concatenates pandas objects along a particular axis with optional set logic along the other axes.

```
import pandas as pd
df1 = pd.DataFrame({'1950':[100, 200, 300], '1955':[300, 400, 500], '1960':[300, 600, 900]}, index =['A', 'B', 'C'])
```

```
df2 = pd.DataFrame({'1965':[500, 600, 900], '1970':[350, 460, 550], '1975':[450, 660, 980]}, index =['A', 'B', 'C'])
```

```
pd.concat([df1, df2])
```

```
pd.concat([df1, df2], axis = 1)
```